In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.model_selection import LeaveOneOut, train_test_split
from sklearn.metrics import accuracy_score
from statistics import mean, stdev
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from matplotlib import image, pyplot
import matplotlib.pyplot as plt
import cv2
import seaborn as sn
import numpy as np
import pandas as pd
import copy as cp

# PCA
1. Zbiory danych użyte w ćwiczeniu:
  * 10-klasowy MNIST
  * 2-klasowy eeg-eye-state (własny)
  * własny obrazek 200x200 pikseli

In [ ]:
mnist = fetch_openml("mnist_784", data_home="./mnist_784", cache=True)

In [ ]:
ees = fetch_openml("eeg-eye-state", data_home="./eeg-eye-state", cache=True)

2. W ćwiczeniu wykorzystamy zbiory testowe stanowiące 10, 25, 75% zbiory danych (po 10 instancji dla każdego przypadku) oraz one-to-all (LeaveOneOut). 

# KNN z błędem

In [ ]:
def error_for_k(dataset, k, train_size, random_seed):

    # dzielimy dane na treningowe i testowe
    data = dataset.data
    labels = dataset.target
    train_data, test_data, train_labels, test_labels = train_test_split(data, labels, train_size=train_size, random_state=random_seed)
    
    # trenujemy klasyfikator
    knn_all = KNeighborsClassifier(n_neighbors = k)
    knn_all.fit(train_data, train_labels)

    prediction = knn_all.predict(test_data)
    # liczymy błąd jako średnią ilości k najbliższych punktów uczących do badanego punktu testowego dzielone przez lczbę k
    dist, indices  = knn_all.kneighbors(test_data)
    d = pd.DataFrame({'prediction': prediction, 'indices': list(indices), 'actual': test_labels})
    d['error'] = d.apply(lambda x: np.count_nonzero(train_labels[x['indices']] == x['actual']) / k, axis=1)
    return d['error'].mean()

def get_score(dataset, k, train_size):
    # mierzymy błąd dla 10 powtórzeń
    errors = [error_for_k(dataset, k, train_size, i) for i in range(10)]
    mean_error = mean(errors)
    mean_stdev = stdev(errors)
    score = (k, train_size, mean_error, mean_stdev)
    print(score)
    return score

samples = 2000 # bo dla większej ilości za długo się liczy
def get_mean_and_stdev(dataset, samples=2000):
    d2 = cp.copy(dataset)
    d2.data, d2.target = dataset.data[:samples], dataset.target[:samples]
    return [get_score(d2, k, train_size) for k in [1, 3, 5, 100] for train_size in [0.05, 0.25, 0.75]]


In [ ]:
print("MNIST 10 klas")
print("k, train_size, mean_error, mean_stdev")
tmp = get_mean_and_stdev(mnist)

In [ ]:
print("Zbiór własny 2 klasy")
print("k, train_size, mean_error, mean_stdev")
tmp = get_mean_and_stdev(ees)

In [ ]:
# Poniżej wyniki z użyciem splittera LeaveOneOut

def build_result_df(train_data, test_data, k, train_labels, test_labels):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(train_data, train_labels)
    
    prediction = knn.predict(test_data)
    dist, ind = knn.kneighbors(test_data)
    d = pd.DataFrame({'prediction': prediction, 'ind': list(ind), 'actual': test_labels})
    d['error'] = d.apply(lambda x: np.count_nonzero(train_labels[x['ind']] == x['actual']) / k, axis=1)
    return d

def leave_one_out_kneighbors(dataset, samples, k):
    splitter = LeaveOneOut()

    data_sample = dataset.data[:samples]
    labels_sample = dataset.target[:samples]

    for train_indices, test_indices in splitter.split(data_sample):
        train_data = data_sample[train_indices]
        test_data = data_sample[test_indices]

        train_labels = labels_sample[train_indices]
        test_labels = labels_sample[test_indices]
        
        yield build_result_df(train_data, test_data, k, train_labels, test_labels)
        
samples = 200 # zmniejszamy, ponieważ liczenie trwa długo dla 2000 próbek
ks = [1,3,5,10] # ilość sąsiadów

def get_errors_for_leaveoneout(dataset, samples, k):
    print("Using %s-nearest-neighbors classifier.:" % k)
    mean= np.mean(np.array([d['error'].mean() for d in leave_one_out_kneighbors(dataset, samples, k)]))
    print("mean: ", mean)

print("zbiór MNIST 10-klasowy (leave one out split):")
for k in ks: 
    get_errors_for_leaveoneout(mnist, samples, k)

print("zbiór 2-klasowy (leave one out split):")
for k in ks:
    get_errors_for_leaveoneout(ees, samples, k)

Zadanie 1.:

Dla zbiorów danych wielowymiarowych zastosować klasyfikator k-NN dla każdych 10 instancji, policzyć średni błąd klasyfikacji oraz jego odchylenie standardowe, dla k=1,3,5,100. Zbiór uczący: 5% , 25% , 75% 

Dla jakiego k otrzymujemy najmniejszy błąd klasyfikacji oraz najmniejsze odchylenie standardowe (wariancję) dla poszczególnych przypadków uczenia i zbiorów danych? Dlaczego? 

Zbiór MNIST

| % zbioru testowego | k   | średni błąd | odchylenie standardowe |   |
|--------------------|-----|-------------|------------------------|---|
| 5%                 | 1   |             |                        |   |
| 5%                 | 3   |             |                        |   |
| 5%                 | 5   |             |                        |   |
| 5%                 | 100 |             |                        |   |
| 25%                | 1   |             |                        |   |
| 25%                | 3   |             |                        |   |
| 25%                | 5   |             |                        |   |
| 25%                | 100 |             |                        |   |
| 75%                | 1   |             |                        |   |
| 75%                | 3   |             |                        |   |
| 75%                | 5   |             |                        |   |
| 75%                | 100 |             |                        |   |

Zbiór własny

| % zbioru testowego | k   | średni błąd | odchylenie standardowe |   |
|--------------------|-----|-------------|------------------------|---|
| 5%                 | 1   |             |                        |   |
| 5%                 | 3   |             |                        |   |
| 5%                 | 5   |             |                        |   |
| 5%                 | 100 |             |                        |   |
| 25%                | 1   |             |                        |   |
| 25%                | 3   |             |                        |   |
| 25%                | 5   |             |                        |   |
| 25%                | 100 |             |                        |   |
| 75%                | 1   |             |                        |   |
| 75%                | 3   |             |                        |   |
| 75%                | 5   |             |                        |   |
| 75%                | 100 |             |                        |   |

# Wizualizacja PCA

In [ ]:
scaler = StandardScaler()

data = ees.data
target = ees.target

train_x, test_x, train_y, test_y = train_test_split(data, target, test_size = 0.05)
scaler.fit(train_x)

#normalizacja cech
train_x = scaler.transform(train_x) 
test_x = scaler.transform(test_x)

In [ ]:
def remove_outliers(x, y, m=2): 
    #Usuwamy te rzędy, w których co najmniej jedna współrzędna jest oddalona od średniej o więcej niż m * wartość_standardowa
    not_outliers = np.all(abs(x - np.mean(x)) < m * np.std(x), axis=1)
    return x[not_outliers], y[not_outliers]

pca = PCA(n_components=2)

reduced_x_with_outliers = pca.fit_transform(train_x)
reduced_x, train_y_2 = remove_outliers(reduced_x_with_outliers, train_y, m=20)
a_indices = train_y_2 == '1'
b_indices = train_y_2 != '1'

#rysujemy na niebiesko / zielono wszystkie punkty treningowe
plt.scatter(reduced_x[a_indices, 0], reduced_x[a_indices, 1], color = "green")
plt.scatter(reduced_x[b_indices, 0], reduced_x[b_indices, 1], color = "blue")

best_k = 3 # tutaj nalezy podać najlepsze k z poprzedniego zadania dla własnego zbioru danych
knn = KNeighborsClassifier(n_neighbors = best_k)
knn.fit(reduced_x_with_outliers, train_y)
red_test_x = pca.transform(test_x)
pred_y = knn.predict(red_test_x)

#rysujemy na czerwono wszystkie błednie określone punkty
wrong_i = pred_y != test_y
plt.scatter(red_test_x[wrong_i, 0], red_test_x[wrong_i, 1], color = "red")

Zadanie 2.:

Zwizualizować przy pomocy PCA wyniki dla zbioru własnego, zakładając że zbiór uczący to 5% oraz 75% oraz najlepsze k wybrane w punkcie 3. Porównać do PCA w którym znana jest przynależność do klas 100% punktów. Przedyskutować wynik. 

# KNeighborsRegressor dla kolorowego obrazka

Należy wygenerować dla obrazka zbiór pikseli uczących stanowiących pewien % oryginalnego obrazka. Wartości pozostałych pikseli zostaną obliczone jako średnia z najbliższych k sąsiadów ze zbioru uczącego.

In [ ]:
# 5. Wczytujemy obrazek w oryginalnym rozmiarze.
cats = image.imread('aoshima-cats.jpg') # 900x900
cats = cv2.resize(cats, (200,200)) # scale to 200x200
pyplot.imshow(cats)

In [ ]:
DIM = 200 # wymiar N badanego obrazka
p = .5 # % zbioru testowego
k = 10 # ile sąsiadów

# tworzymy tablicę indeksów (N^2 x 2)
indices_flat = np.array([(x,y) for x in range(DIM) for y in range(DIM)])

# spłaszczamy obrazek do tego samego wymiaru (N^2 x 3 rgb)
cats_flat = np.reshape(cats, (DIM*DIM, -1))

# wybór zbioru pikseli i indeksów trenujących i testowych
train_c, test_c, train_i, test_i = train_test_split(cats_flat, indices_flat, test_size=p)

# korzystamy z metody najbliższych sąsiadów i uczymy model
knr = KNeighborsRegressor(n_neighbors=k)
knr.fit(train_i, train_c)

# predykcja kolorów pozostałych pixeli (zaokrąglamy do liczby całkowitej z przedziału [0..255])
predict_c = knr.predict(test_i).astype(np.int32)

In [ ]:
# łączymy piksele oryginalne z tymi z predykcji
pixels = np.concatenate((train_c, predict_c))
indices = np.concatenate((train_i, test_i))
indices_flattened = np.array([x[0]*DIM + x[1] for x in indices])

result_image = pixels[np.argsort(indices_flattened)]

Jak wyglądają wygenerowane obrazki? Możemy sprawdzić jak modyfikacje parametrów p i k wpływają na finalny rezultat.

In [ ]:
# uzyskany obrazek
def reshape_and_display(image, DIM=200):
    pyplot.imshow(image.reshape((DIM, DIM, -1)))

reshape_and_display(result_image)

Do oszacowania błędu wykorzystujemy średni błąd kwadratowy wartości pikseli RGB w porównaniu z oryginałem.

In [ ]:
# 6. błąd w porównaniu z oryginałem 
from sklearn.metrics import mean_squared_error
mean_squared_error(cats_flat, result_image)

Następnie, powtarzamy powyższą procedurę kilkukrotnie (Q=5) i generujemy obrazek, którego wartość pikseli obliczamy jako średnią z wygenerowanych Q obrazków. Sprawdzamy czy zmniejszył się błąd. Jak teraz wygląda obrazek?

In [ ]:
# 7. wykonujemy kilka prób i generujemy obrazek będący średnią z wartości pixeli
def predicted_image(original_image, p, k,DIM=200):
    indices_flat = np.array([(x,y) for x in range(DIM) for y in range(DIM)])
    image_flat = np.reshape(original_image, (DIM*DIM, -1))
    
    train_c, test_c, train_i, test_i = train_test_split(image_flat, indices_flat, test_size=p)
    
    knr = KNeighborsRegressor(n_neighbors=k)
    knr.fit(train_i, train_c)
    predict_c = knr.predict(test_i).astype(np.int32)
    
    pixels = np.concatenate((train_c, predict_c))
    indices = np.concatenate((train_i, test_i))
    indices_flattened = np.array([x[0]*DIM + x[1] for x in indices])

    return pixels[np.argsort(indices_flattened)]
    
Q = 5 # ilość prób
predicted_images = np.array([predicted_image(cats, p, k) for _ in range(Q)])
mean_image = np.mean(predicted_images, axis=0, dtype='uint64')

In [ ]:
# obrazek powstały ze średniej
reshape_and_display(mean_image)

In [ ]:
# Czy bład się zmniejszył? otóż tak
mean_squared_error(cats_flat, mean_image)

Zadanie 3.:

a. 
Dla obrazka wygenerować losowo zbiór pikseli uczących w ilości 10%, 50%, 75%. Zrobić to 5- krotnie (5
instancji dla każdego zbioru uczącego). Kolory reszty z pikseli policzyć jako średnią z k=1, 3, 10
najbliższych sąsiadów ze zbioru uczącego. Jak wyglądają wygenerowane obrazki? (Podpowiedź: można użyć funkcji predicted_image oraz reshape_and_display, odpowiednio sparametryzowanej) 

b.
Znaleźć błąd dla każdego wygenerowanego obrazka w porównaniu z oryginałem. 
(można użyć funkcji mean_squared_error i wykorzystać tabelkę poniżej na wyniki)

c.
Dla każdego przypadku uczącego (10%, 50%, 75%.) policzyć średnie wartości pikseli z wygenerowanych 5 obrazków. Jak wygląda tak wygenerowany obrazek. Czy błąd się
zmniejszył??? 

#### Błędy dla podpunktu 3.b

|                    | k najbliższych sąsiadów |     |      |   |
|:------------------:|:-----------------------:|:---:|:----:|:-:|
| % zbioru testowego |           k=1           | k=3 | k=10 |   |
|         10%        |                         |     |      |   |
|         50%        |                         |     |      |   |
|         75%        |                         |     |      |   |


#### Błędy dla podpunktu 3.c

|                    | k najbliższych sąsiadów |     |      |   |
|:------------------:|:-----------------------:|:---:|:----:|:-:|
| % zbioru testowego |           k=1           | k=3 | k=10 |   |
|         10%        |                         |     |      |   |
|         50%        |                         |     |      |   |
|         75%        |                         |     |      |   |

In [ ]:
# MIEJSCE NA KOD DO ZADANIA 3.
